In [1]:
import math
import numpy as np
import pandas as pd
# import pandas_datareader as web
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.linear_mode
#import LinearRegression
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras import Model
from keras.layers import Dense, LSTM, Dropout, Concatenate
from keras.optimizers import Adam
import matplotlib.pyplot as plt

#plt.style.use('fivethirtyeight')
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-dark')

2024-05-03 19:40:35.856066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
demo_df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv');
demo_df

,date,open,low,high,volume,close,Name,Date,Positve,Negative,Neutral,Total,NeutralPos,NeutralNeg
0,10/2/2017,964.00,952.1201,967.305,2415846,959.19,AMZN,10/2/2017,37.088734,7.577268,55.333998,1003,37.088734,14.260219
1,10/3/2017,958.00,950.3700,963.690,2643484,957.10,AMZN,10/3/2017,38.159879,11.085973,50.754148,1326,38.159879,22.348774
2,10/4/2017,954.21,954.0500,967.790,2460721,965.45,AMZN,10/4/2017,36.168582,10.727969,53.103448,1305,36.168582,19.641470
3,10/5/2017,970.00,969.6400,981.510,3119487,980.85,AMZN,10/5/2017,47.939017,7.227555,44.833427,1771,47.939017,24.411076
4,10/6/2017,975.64,975.6400,995.750,3719840,989.58,AMZN,10/6/2017,34.628045,11.586570,53.785385,1519,34.628045,19.818134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3/5/2018,1494.24,1481.0000,1525.380,5233934,1523.61,AMZN,3/5/2018,38.274182,7.794015,53.931802,1437,38.274182,15.879162
105,3/6/2018,1533.20,1528.0000,1542.130,4561718,1537.64,AMZN,3/6/2018,45.234604,6.818182,47.947214,1364,45.234604,20.887916
106,3/7/2018,1526.52,1522.5100,1545.900,4174123,1545.00,AMZN,3/7/2018,38.167260,9.786477,52.046263,1124,38.167260,19.757162
107,3/8/2018,1550.00,1545.2500,1554.880,3512528,1551.86,AMZN,3/8/2018,37.221728,8.726625,54.051647,1123,37.221728,16.691926


In [3]:
df_decisions = pd.read_csv('./stock_decisions.csv');
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Sector of stock,Buy/Sell/Keep,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg
0,964.00,957.550,13.32,2.414993,5.338076,IT,Buy,37.088734,7.577268,1003,55.333998,37.088734,14.260219
1,958.00,959.830,13.74,3.344313,19.608973,IT,Buy,38.159879,11.085973,1326,50.754148,38.159879,22.348774
2,954.21,975.425,11.87,2.726208,13.568787,IT,Buy,36.168582,10.727969,1305,53.103448,36.168582,19.641470
3,970.00,982.610,20.11,0.544740,9.648946,IT,Buy,47.939017,7.227555,1771,44.833427,47.939017,24.411076
4,975.64,992.115,11.00,4.698582,6.444096,IT,Buy,34.628045,11.586570,1519,53.785385,34.628045,19.818134


In [4]:
df_decisions.drop('Sector of stock', axis=1, inplace=True)

In [5]:
input = []

todays_date = '2/16/2018'
list_of_stocks = []
list_of_stocks.append({'Percentage of portfolio': 2.5, 'Percentage of sector': 15.56, 'Sector of Stock': 'IT', 'comp': 'AMZN'});
list_of_stocks.append({'Percentage of portfolio': 4.5, 'Percentage of sector': 7.8, 'Sector of Stock': 'IT', 'comp': 'MSFT'});
list_of_stocks.append({'Percentage of portfolio': 2.5, 'Percentage of sector': 17.26, 'Sector of Stock': 'Health', 'comp': 'PFIZER'});
list_of_stocks

[{'Percentage of portfolio': 2.5,
  'Percentage of sector': 15.56,
  'Sector of Stock': 'IT',
  'comp': 'AMZN'},
 {'Percentage of portfolio': 4.5,
  'Percentage of sector': 7.8,
  'Sector of Stock': 'IT',
  'comp': 'MSFT'},
 {'Percentage of portfolio': 2.5,
  'Percentage of sector': 17.26,
  'Sector of Stock': 'Health',
  'comp': 'PFIZER'}]

In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [7]:
from tensorflow.keras.losses import MeanSquaredLogarithmicError, MeanSquaredError
from tensorflow.keras.saving import register_keras_serializable
@register_keras_serializable(package="MyLoss")
def MyLoss(y_true, y_pred):
    msle = MeanSquaredLogarithmicError()(y_true[:,0], y_pred[:,0])
    mse = MeanSquaredError()(y_true[:,1],y_pred[:,1])
    return  msle + mse

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
def makeNextDayVector(filtered_df, todays_date, stock):
    
    
    filtered_df['diff']=filtered_df['high']-filtered_df['low']
    filtered_df['exp']=(filtered_df['open']+filtered_df['close'])/2
    
       
    
    loaded_model = load_model('model')
    curr_row = filtered_df[filtered_df['Date'] == todays_date]
    curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    filtered_df.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    curr_stock_value = curr_row['open'].iloc[0]
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    filtered_df[filtered_df.columns] = scaler.fit_transform(filtered_df[filtered_df.columns])
    
    predicted_values = pd.DataFrame()
    
    predicted_values[['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg']] = 0
    
    predicted_values[['diff', 'exp']] = loaded_model.predict(np.array([filtered_df.values]).astype(float))
    # predicted_values=scaler.inverse_transform(predicted_values)
    print(predicted_values.columns)
    
    predicted_values[predicted_values.columns] = scaler.inverse_transform(predicted_values)
    
    
    predicted_values.drop(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg'], axis=1, inplace=True)
    
    
    expected_value = predicted_values['exp'].iloc[0]
    error = predicted_values['diff'].iloc[0]
    
    percen_portfolio = stock['Percentage of portfolio']
    percen_sector = stock['Percentage of sector']
    sector = stock['Sector of Stock']
    
    positives = curr_row['Positve'].iloc[0]
    negatives = curr_row['Negative'].iloc[0]
    total = curr_row['Total'].iloc[0]
    neutral = curr_row['Neutral'].iloc[0]
    neutral_pos = curr_row['NeutralPos'].iloc[0]
    neutral_neg = curr_row['NeutralNeg'].iloc[0]
    
    
    
    stock_vect = []
    stock_vect.append(curr_stock_value)
    stock_vect.append(expected_value)
    stock_vect.append(error)
    stock_vect.append(percen_portfolio)
    stock_vect.append(percen_sector)
    stock_vect.append(positives)
    stock_vect.append(negatives)
    stock_vect.append(total)
    stock_vect.append(neutral)
    stock_vect.append(neutral_pos)
    stock_vect.append(neutral_neg)
    
    # add posi negi from filtered data in stock vector
    
    # stock_vect.append(sector)
    
    
    return stock_vect
    

In [9]:
import logging
logging.basicConfig(level=logging.INFO)

In [10]:
next_vectors = []
for i in range(len(list_of_stocks)):
    if(list_of_stocks[i]['comp'] == 'AMZN'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv');
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'AMZN'})
    elif(list_of_stocks[i]['comp'] == 'AAPL'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AAPLStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'AAPL'})
    elif(list_of_stocks[i]['comp'] == 'MSFT'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/MSFTStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'MSFT'})
    elif(list_of_stocks[i]['comp'] == 'PFIZER'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/PfizerStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'PFIZER'})


2024-05-03 19:40:38.576011: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-05-03 19:40:39.051737: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2024-05-03 19:40:39.051837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2024-05-03 19:40:39.0

1/1 [==============================] - 0s 491ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


2024-05-03 19:40:44.276296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2024-05-03 19:40:44.276419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2024-05-03 19:40:44.276479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 562ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


2024-05-03 19:40:49.598497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2024-05-03 19:40:49.598584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2024-05-03 19:40:49.598653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 485ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


In [11]:
next_vectors[0]['next_day_vector']

[1457.37,
 1490.1939747652411,
 57.293122385740276,
 2.5,
 15.56,
 35.70658037,
 12.29773463,
 927,
 51.99568501,
 35.70658037,
 22.31899822]

In [12]:
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Buy/Sell/Keep,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg
0,964.00,957.550,13.32,2.414993,5.338076,Buy,37.088734,7.577268,1003,55.333998,37.088734,14.260219
1,958.00,959.830,13.74,3.344313,19.608973,Buy,38.159879,11.085973,1326,50.754148,38.159879,22.348774
2,954.21,975.425,11.87,2.726208,13.568787,Buy,36.168582,10.727969,1305,53.103448,36.168582,19.641470
3,970.00,982.610,20.11,0.544740,9.648946,Buy,47.939017,7.227555,1771,44.833427,47.939017,24.411076
4,975.64,992.115,11.00,4.698582,6.444096,Buy,34.628045,11.586570,1519,53.785385,34.628045,19.818134


In [13]:
# make but keep and sell dfs

# buy df
buy_df = df_decisions[df_decisions['Buy/Sell/Keep'] == 'Buy']

sell_df = df_decisions[df_decisions['Buy/Sell/Keep'] == 'Sell']

keep_df = df_decisions[df_decisions['Buy/Sell/Keep'] == 'Keep']

buy_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
sell_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
keep_df.drop(columns=['Buy/Sell/Keep'], inplace=True)

/tmp/ipykernel_157936/280646284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
/tmp/ipykernel_157936/280646284.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
/tmp/ipykernel_157936/280646284.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_df.drop(columns=['Buy/Sell/Keep'], inplace=True)


In [14]:
buy_df.values[0]

array([ 964.        ,  957.55      ,   13.32      ,    2.41499291,
          5.33807605,   37.0887338 ,    7.5772682 , 1003.        ,
         55.33399801,   37.0887338 ,   14.26021879])

In [15]:
next_vectors

[{'next_day_vector': [1457.37,
   1490.1939747652411,
   57.293122385740276,
   2.5,
   15.56,
   35.70658037,
   12.29773463,
   927,
   51.99568501,
   35.70658037,
   22.31899822],
  'comp': 'AMZN'},
 {'next_day_vector': [92.45,
   92.62311322450637,
   3.3932703372836044,
   4.5,
   7.8,
   40.5362776,
   12.30283912,
   634,
   47.16088328,
   40.5362776,
   12.35454261],
  'comp': 'MSFT'},
 {'next_day_vector': [35.76,
   36.613284439742564,
   1.1120461332768177,
   2.5,
   17.26,
   30.63583815,
   8.670520231,
   173,
   60.69364162,
   30.63583815,
   12.93880494],
  'comp': 'PFIZER'}]

In [16]:
# similarty 
from sklearn.metrics.pairwise import cosine_similarity

buy_score = []
sell_score = []
keep_score = []

for i in range(len(next_vectors)):
    buy_score_similarity = 0
    c = len(buy_df)
    buy_score_similarity = cosine_similarity(buy_df.values, [next_vectors[i]['next_day_vector']])
    total_cos_buy_score = np.sum(buy_score_similarity)
    buy_score_similarity/=c
    buy_score.append({'buyscore': total_cos_buy_score, 'comp': next_vectors[i]['comp']})
    
    
    sell_score_similarity = 0
    c = len(sell_df)
  
    sell_score_similarity = cosine_similarity(sell_df.values, [next_vectors[i]['next_day_vector']])
    sell_score_similarity/=c
    total_cos_sell_score = np.sum(sell_score_similarity)
    sell_score.append({'sellscore': total_cos_sell_score, 'comp': next_vectors[i]['comp']})
    
    keep_score_similarity = 0
    c = len(keep_df)
    
    keep_score_similarity = cosine_similarity(keep_df.values, [next_vectors[i]['next_day_vector']])
    keep_score_similarity/= c
    total_cos_keep_score = np.sum(keep_score_similarity)
    keep_score.append({'keepscore': total_cos_keep_score, 'comp': next_vectors[i]['comp']})



In [17]:
top_n = 2

sorted_buy = sorted(buy_score, key=lambda x: x['buyscore'])
sorted_sell = sorted(sell_score, key=lambda x: x['sellscore'])
sorted_keep = sorted(keep_score, key=lambda x: x['keepscore'])

print(sorted_sell)
print(sorted_buy)
print(sorted_keep)

top_buy = sorted_buy[:top_n]
top_sell = sorted_sell[:top_n]
top_keep = sorted_keep[:top_n]


[{'sellscore': 0.6132708921582581, 'comp': 'AMZN'}, {'sellscore': 0.8309832194922675, 'comp': 'MSFT'}, {'sellscore': 0.9131130393041016, 'comp': 'PFIZER'}]
[{'buyscore': 337.7810335359952, 'comp': 'AMZN'}, {'buyscore': 484.47017835758436, 'comp': 'PFIZER'}, {'buyscore': 506.62293949711676, 'comp': 'MSFT'}]
[{'keepscore': 0.5899467893557104, 'comp': 'AMZN'}, {'keepscore': 0.9014277854944079, 'comp': 'MSFT'}, {'keepscore': 0.917537919227701, 'comp': 'PFIZER'}]


In [22]:
# Assuming sorted_buy, sorted_sell, and sorted_keep contain tuples of (score, company)
top_buy = sorted_buy[:top_n][::-1]

top_buy_stocks = set(stock for stock in top_buy)

filtered_sell = [sell_stock for sell_stock in sorted_sell if sell_stock not in top_buy_stocks]

filtered_sell = sorted(sell_score, key=lambda x: x['sellscore'])

top_sell_filtered = filtered_sell[:top_n][::-1]

top_sell = top_sell_filtered

top_sell_stocks = set(stock for stock in top_sell)

filtered_keep = [keep_stock for keep_stock in sorted_keep if keep_stock not in top_sell_stocks]

filtered_keep = sorted(keep_score, key=lambda x: x['keepscore'])
top_keep_filtered = filtered_keep[:top_n][::-1]
top_keep = top_keep_filtered



print(top_buy)
print(top_sell)
print(top_keep)


TypeError: unhashable type: 'dict'

In [19]:
print('Stocks to buy: ')
for i in range(len(top_buy)):
    print(top_buy[i]['comp'])

Stocks to buy: 
PFIZER
AMZN


In [20]:
print('Stocks to Sell')

for i in range(len(top_sell)):
    print(top_sell[i]['comp'])

Stocks to Sell
MSFT
AMZN


In [21]:
print('Stocks to Keep')

for i in range(len(top_keep)):
    print(top_keep[i]['comp'])

Stocks to Keep
MSFT
AMZN
